### Instalamos plotly, que usaremos para graficar

In [2]:
pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [plotly]2m1/2 [plotly]
Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 86.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]
Note: you may need to restart the kernel to use updated packages.


### Importamos las librerias que usaremos e iniciamos la sesion de spark

In [3]:
from pyspark.sql import SparkSession
import plotly.graph_objs as go
from plotly.offline import plot

print("Estadsticas de recetas utilizando spark-docker")

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

Estadsticas de recetas utilizando spark-docker


25/07/28 18:54:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Leemos el archivo:

In [4]:
data = spark.read.csv(path="data/recetas_2025.csv", sep=",", header=True)

### Corroboramos la data structure

In [5]:
print("Archivo cargado.", data.count(), "lineas,",len(data.columns),"columnas.")
data.show(20)

Archivo cargado. 465203 lineas, 18 columnas.
+----+---+--------------------------+---------------+-----------+----+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+------------------+----------------+--------------+-------------------------+
| any|mes|codi de la regió sanitària|regió sanitària|grup d'edat|sexe|codi del grup ATC nivell 1|   grup ATC nivell 1|codi del grup ATC nivell 2|   grup ATC nivell 2|codi del grup ATC nivell 3|   grup ATC nivell 3|codi del grup ATC nivell 4|   grup ATC nivell 4|nombre de receptes|nombre d'envasos|import íntegre|import aportació CatSalut|
+----+---+--------------------------+---------------+-----------+----+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+--------------------------+--------------------+-------------

### Seleccionamos un subset de los datos

In [7]:
data = data.select("regió sanitària", "sexe", "codi del grup ATC nivell 3", "nombre de receptes")
data.show(20)

+---------------+----+--------------------------+------------------+
|regió sanitària|sexe|codi del grup ATC nivell 3|nombre de receptes|
+---------------+----+--------------------------+------------------+
|         LLEIDA|Dona|                      A01A|                11|
|         LLEIDA|Dona|                      A02B|                 2|
|         LLEIDA|Dona|                      A07A|                 2|
|         LLEIDA|Dona|                      A07C|                32|
|         LLEIDA|Dona|                      A11C|               292|
|         LLEIDA|Dona|                      A11H|                 1|
|         LLEIDA|Dona|                      B03B|                 2|
|         LLEIDA|Dona|                      B03B|                 1|
|         LLEIDA|Dona|                      B05B|                30|
|         LLEIDA|Dona|                      C01C|                 1|
|         LLEIDA|Dona|                      C07A|                 5|
|         LLEIDA|Dona|            

### Agregamos alguna informacion interesante

In [16]:
data.groupby("codi del grup ATC nivell 3","sexe","nombre de receptes").agg({'nombre de receptes': 'sum'}).orderBy("codi del grup ATC nivell 3").show(10)

+--------------------------+----+------------------+-----------------------+
|codi del grup ATC nivell 3|sexe|nombre de receptes|sum(nombre de receptes)|
+--------------------------+----+------------------+-----------------------+
|                      A01A|Dona|                14|                  182.0|
|                      A01A|Home|                18|                   36.0|
|                      A01A|Dona|                 2|                  382.0|
|                      A01A|Dona|                41|                   41.0|
|                      A01A|Dona|                 3|                  357.0|
|                      A01A|Dona|                47|                   47.0|
|                      A01A|Home|                60|                   60.0|
|                      A01A|Home|                45|                   45.0|
|                      A01A|Home|                39|                   39.0|
|                      A01A|Dona|                11|                  319.0|

AnalysisException: cannot resolve '`sum(nombre de receptes)`' given input columns: [codi del grup ATC nivell 3, nombre de receptes, regió sanitària, sexe];;
'Sort ['sum(nombre de receptes) ASC NULLS FIRST], true
+- Project [regió sanitària#19, sexe#21, codi del grup ATC nivell 3#26, nombre de receptes#30]
   +- Relation[any#16,mes#17,codi de la regió sanitària#18,regió sanitària#19,grup d'edat#20,sexe#21,codi del grup ATC nivell 1#22,grup ATC nivell 1#23,codi del grup ATC nivell 2#24,grup ATC nivell 2#25,codi del grup ATC nivell 3#26,grup ATC nivell 3#27,codi del grup ATC nivell 4#28,grup ATC nivell 4#29,nombre de receptes#30,nombre d'envasos#31,import íntegre#32,import aportació CatSalut#33] csv


In [19]:
# Agregamos por sexo
data = data.groupby("sexe").agg({'sum(nombre de receptes)': 'sum'})
data.show(10)

+-----------------+----------------------------+
|             sexe|sum(sum(nombre de receptes))|
+-----------------+----------------------------+
|             Dona|                 3.8935871E7|
|             Home|                 2.9600307E7|
|           Altres|                     25077.0|
|Sense especificar|                    300806.0|
+-----------------+----------------------------+



### Graficamos

In [23]:
dataFrame = data.toPandas()

# Crear gráfico de barras
fig = go.Figure(go.Bar(
    x=dataFrame['sexe'],
    y=dataFrame['sum(sum(nombre de receptes))'],
    marker_color='indigo'
))
fig.update_layout(title="Recetas totales emitidas por sexo", xaxis_title="Manufacturer", yaxis_title="Num. Recetas")
plot(fig, filename='recetas_por_sexo.html')

'recetas_por_sexo.html'

### Por ultimo detenemos Spark

In [24]:
spark.stop()